# Bắt Đầu với PyTorch

## PyTorch là gì?

- PyTorch là một thư viện Python (Là một wrapper của Torch - Thư viện C)
    + Thay thế cho các thư viện tính toán khoa học (scientific computing framework, như Numpy) có khả năng kết hợp với GPU
    + Cung cấp nền tảng đơn giản cho việc nghiên cứu Deep Learning
    
- Những ai đang sử dụng PyTorch
    + Trong công nghiệp: Facebook, Twitter, NVIDIA, SaleForce, ...
    + Trong nghiên cứu: CMU, Stanford, ...
    + Ngày càng được ưa chuộng và dùng nhiều hơn

Tutorial này giới thiệu các cài đặt PyTorch và dùng PyTorch để xây dựng một ứng dụng Deep Learning

## Cài đặt PyTorch

Cách cài đặt PyTorch đơn giản nhất là sử dụng Anaconda Python. Chạy lệnh sau trong một môi trường đã tạo:

`conda install pytorch torchvision -c pytorch`

## PyTorch "Hello World"

Tiếp theo chúng ta sẽ sử dụng PyTorch để xây dựng ứng dụng "Hello World" của Deep Learning - Xây dựng mạng neuron để nhận dạng chữ số viết tay từ tập dữ liệu MNIST (Một tập dữ liệu chuẩn trong ML, thị giác máy tính (Computer Vision)).

Chúng ta có thể tải dữ tập dữ liệu này từ [MNIST Website](http://yann.lecun.com/exdb/mnist/) và load dữ liệu PyTorch. Tuy nhiên để đơn giản hoá qui trình cho những nhà nghiên cứu và phát triền, PyTorch đã đóng gói những tập dữ liệu này thành những dữ liệu chuẩn trong thư viện.

### Load dữ liệu

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms),
    batch_size=64, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms),
    batch_size=64, shuffle=True
)

Đoạn code trên tạo 2 đối tượng DataLoader (train_loader, test_loader) để download dữ liệu theo batch (64 ảnh mỗi batch) từ dữ liệu 60000 ảnh MNIST và lưu vào file train và test. Ngoài ra gói torchvision còn giúp chúng ta tạo được những pipeline phức tạp để xử lý dữ liệu như cropping, xoay ảnh, scale ảnh, .... Trong ví dụ trên chúng ta load ảnh MNIST (28x28 pixels) chuyển thành tensor (1x28x28) sau đấy thì chuẩn hoá giá trị pixels từ giá trị từ 0...255 về giá trị từ -1...1 băng cách định nghĩa lớp Normalize với mean (0.1307) và standard deviation (0.3081) của toàn bộ tập dữ liệu MNIST. Việc chuẩn hoá dữ liệu sẽ giúp việc training mạng neural tốt hơn rất nhiều.

### Định nghĩa mạng Neuron
Tiếp theo chúng ta sẽ tạo một mạng neural sử dụng Pytorch nn.Module

In [4]:
class HelloWorldNet(nn.Module):
    def __init__(self, image_size):
        super(HelloWorldNet, self).__init__()
        self.image_size = image_size
        self.fc0 = nn.Linear(image_size, 1000)
        self.fc1 = nn.Linear(1000, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = x.view(-1, self.image_size)
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)

Để xây dựng một mạng neural, chúng ta tạo một lớp kế thừa từ nn.Module và định nghĩa hàm khởi tạo và hàm `forward()`.

Chúng ta định nghĩa tất cả các lớp của mạng neuron trong hàm khởi tạo, ở ví dụ trên chúng ta tạo một mạng neuron đơn giản với 3 fully-connected layers, sử dụng hàm tuyến tính. 

Hàm `forward()` định nghĩa cách tính toán giá trị đầu ra cho mỗi layer. Đầu tiên chúng ta phải đổi tensor của ảnh (1x28x28) thành một dang có chiều phù hợp cho lớp đầu tiên của mạng neuron. Hàm `view()` giúp chúng ta làm điều này. Trong ví dụ trên tensor sẽ được dàn (flattens) thành một tensor có chiều là 1x784. Các dòng tiếp theo định nghĩa "activation function" cho các layers, trong ví dụ chúng ta sử dụng ReLu (Rectified Linear Unit), ở lớp cuối cùng chúng ta dùng softmax. 

Tiếp theo chúng ta có thể tạo model bằng cách dùng lớp trên:

In [22]:
model = HelloWorldNet(image_size=28*28)

Nếu chúng ta đang làm việc trên máy có GPU, chúng ta cho thể chuyển model sang chế độ chạy trên GPU bằng cách goi hàm `cuda()`:

In [23]:
if torch.cuda.is_available():
    model.cuda()

In [24]:
print(model)

HelloWorldNet(
  (fc0): Linear(in_features=784, out_features=1000, bias=True)
  (fc1): Linear(in_features=1000, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


### Traning and Testing

Sau khi định nghĩa mạng neuron, chúng ta phải "dạy" model sử dụng dữ liệu học. 

In [25]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

def train(epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        data, labels = Variable(data), Variable(labels)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, labels)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Eporch: {} [{}/{} ({:.0f}%)\t Loss: {:.6f}]'.format(epoch, batch_idx * len(data), len(train_loader.dataset),
                                                                            100. * batch_idx / len(train_loader), loss.data[0]))

Trước khi chúng ta tạo hàm `train()` chúng ta phải khởi tạo một "optimizer", optimizer có nhiệm vụ điều chỉnh tham số (parameters) của mỗi layer ở mỗi bước chúng ta train model với một batch dữ liệu. Có nhiều thuật toán optimizer như [RMSProp, AdaGrad, Adam ...](http://ruder.io/optimizing-gradient-descent/index.html). Thuật toán tối ưu được dùng phổ biết nhất hiện này là Adam. Trong ví dụ trên chúng ta sử dụng một thuật toán đơn giản [Stochastic Gradient Descent](http://ruder.io/optimizing-gradient-descent/index.html#stochasticgradientdescent).

trong hàm `train()` chúng ta load dữ liệu ở mỗi batch từ tập dữ liệu học, model sẽ tính đâu ra và so sánh với giá trị thực tế thông qua hàm loss. Sau đấy chúng ta gọi một hàm mà người ta gọi là PyTorch magic, loss.backward(). Hàm này sẽ tự động tính "backpropagation" để đưa ra giá tị update cho tham số, sau cùng chúng ta điều chỉnh tham số thông qua hàm `step()`.

Để đánh giá độ chính xác của model chúng ta tạo hàm `test` để tính độ chính xác bằng cách chạy model trên dữ liệu test 

In [26]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [27]:
for epoch in range(1, 1 + 1):
    train(epoch)
    test()

/Users/trang/anaconda/envs/jupyterlab/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


Train Eporch: 1 [0/60000 (0%)	 Loss: 2.308312]
Train Eporch: 1 [6400/60000 (11%)	 Loss: 2.301066]
Train Eporch: 1 [12800/60000 (21%)	 Loss: 2.250018]
Train Eporch: 1 [19200/60000 (32%)	 Loss: 2.230992]
Train Eporch: 1 [25600/60000 (43%)	 Loss: 2.251629]
Train Eporch: 1 [32000/60000 (53%)	 Loss: 2.212543]
Train Eporch: 1 [38400/60000 (64%)	 Loss: 2.110949]
Train Eporch: 1 [44800/60000 (75%)	 Loss: 2.184118]
Train Eporch: 1 [51200/60000 (85%)	 Loss: 2.130153]
Train Eporch: 1 [57600/60000 (96%)	 Loss: 2.051203]

Test set: Average loss: 2.0537, Accuracy: 3924/10000 (39%)



Chúng ta có được độ chính xác trên tập test đạt khoảng 82% sau 10 epoches. Not bad! cho một model đơn giản.

## Dùng Convolutions Neural Network cho kết quả tốt hơn

In [28]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [29]:
model = CNNNet()
optimizer = optim.SGD(model.parameters(), lr=0.001)
if torch.cuda.is_available():
    model.cuda()
for epoch in range(1, 1 + 1):
    train(epoch)
    test()

Train Eporch: 1 [0/60000 (0%)	 Loss: 2.311843]
Train Eporch: 1 [6400/60000 (11%)	 Loss: 2.301671]
Train Eporch: 1 [12800/60000 (21%)	 Loss: 2.290457]
Train Eporch: 1 [19200/60000 (32%)	 Loss: 2.297308]
Train Eporch: 1 [25600/60000 (43%)	 Loss: 2.274854]
Train Eporch: 1 [32000/60000 (53%)	 Loss: 2.291910]
Train Eporch: 1 [38400/60000 (64%)	 Loss: 2.267767]
Train Eporch: 1 [44800/60000 (75%)	 Loss: 2.255762]
Train Eporch: 1 [51200/60000 (85%)	 Loss: 2.256258]
Train Eporch: 1 [57600/60000 (96%)	 Loss: 2.266452]

Test set: Average loss: 2.2526, Accuracy: 3245/10000 (32%)



Chúng ta có được độ chính xác là 91%, tốt hơn khá nhiều so với HelloWord net.

In [32]:
class BetterCNNNet(nn.Module):
    def __init__(self):
        super(BetterCNNNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d(p=0.25)
        self.fc1 = nn.Linear(64*12*12, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 64*12*12)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.25, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [33]:
model = BetterCNNNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
if torch.cuda.is_available():
    model.cuda()
for epoch in range(1, 1 + 1):
    train(epoch)
    test()

Train Eporch: 1 [0/60000 (0%)	 Loss: 2.287128]
Train Eporch: 1 [6400/60000 (11%)	 Loss: 0.251294]
Train Eporch: 1 [12800/60000 (21%)	 Loss: 0.067553]
Train Eporch: 1 [19200/60000 (32%)	 Loss: 0.110949]
Train Eporch: 1 [25600/60000 (43%)	 Loss: 0.019545]
Train Eporch: 1 [32000/60000 (53%)	 Loss: 0.050329]
Train Eporch: 1 [38400/60000 (64%)	 Loss: 0.034056]
Train Eporch: 1 [44800/60000 (75%)	 Loss: 0.073841]
Train Eporch: 1 [51200/60000 (85%)	 Loss: 0.204919]
Train Eporch: 1 [57600/60000 (96%)	 Loss: 0.026304]

Test set: Average loss: 0.0570, Accuracy: 9816/10000 (98%)



## Học học nữa học mãi
More example: http://pytorch.org/tutorials/

## Learning by doing and earning
Participate in Kaggle competitions: https://www.kaggle.com/competitions